In [1]:
import os
import sys
current_directory = os.getcwd().replace('notebooks', '')
sys.path.append(current_directory)

import time, datetime
import polars as pl
from numpy import mean, std
import json

In [2]:
from utils.helper import read_json
from utils.graph_reader import read_graph_instance, read_rakaj_graph_instance, read_graph_instance_networkrepo
from utils.graph import Graph

from main.ilp_model import ilp_mwdds
from main.greedy_algorithm import greedy_mwdds
from main.genetic_algorithm import GeneticAlgorithm, fitness_func

test_instances_pth = '../instances/test_instances/'
test_instances = os.listdir(test_instances_pth)
test_instances = [test_instances_pth + instance for instance in test_instances]

instances_s_pth = '../instances/small/'
instances_m_pth = '../instances/medium/'
instances_l_pth = '../instances/large/'
instances_xl_pth = '../instances/xlarge/'
instances_s_dict = read_json('../instances/small/small_instances.json')
instances_m_dict = read_json('../instances/medium/medium_instances.json')
instances_l_dict = read_json('../instances/large/large_instances.json')
instances_xl_list = os.listdir(instances_xl_pth)

In [3]:
def save_results_to_csv(results, output_csv):
    df = pl.DataFrame({
        'instance': results['instance'],
        'ga_mean': results['ga_mean'],
        'ga_max': results['ga_max'],
        'ga_std': results['ga_std'],
        'ga_time': results['ga_time'],
        'greedy': results['greedy'],
        'greedy_time': results['greedy_time'],
        'ilp': results['ilp'],
        'ilp_time': results['ilp_time']
    })

    df.write_csv(output_csv)
    print(f'Results saved to {output_csv}')

def save_params_to_json(output_file, params):
    with open(output_file, 'w') as f:
        json.dump(params, f, indent=4)
    
    print(f'Parameters saved to {output_file}')

In [ ]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":100,  "population_size":100, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":20,
    "percent_to_remove": 0.15, "n_elite": 2, "tournament_size": 3, "two_point_crossover_prob": 0.8
}

for instance in test_instances:
        print(f'Running instance {instance}...')
        vertices_w, edges = read_graph_instance(instance)

        results['instance'].append("test")

        ga_results = []
        ga_time = []
        graph = Graph()
        graph.create_from_data(vertices_w, edges)
        
        for i in range(10):
            gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=30, **params)
            gen_algo.run()
            ga_results.append(gen_algo.previous_best_fitness)
            ga_time.append(gen_algo.running_time)
            print(f'Finished {i+1} of 10 in {gen_algo.running_time} s')
    
        results['ga_mean'].append(round(mean(ga_results),6))
        results['ga_max'].append(max(ga_results))
        results['ga_std'].append(round(std(ga_results), 6))
        results['ga_time'].append(round(mean(ga_time), 4))

        start = time.time()
        greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
        results['greedy'].append(greedy_w)
        results['greedy_time'].append(round(time.time() - start, 4))
        
        start = time.time()
        ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
        results['ilp'].append(ilp_w)
        results['ilp_time'].append(round(time.time() - start, 4))

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%H-%M_%d-%m-%Y")
output_csv = f'../results/new/test_results_{formatted_time}.csv'
output_params = f'../results/new/test_ga_params_{formatted_time}.json'

save_results_to_csv(results, output_csv)
save_params_to_json(output_params, params)

In [4]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":300,  "population_size":1000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":50,
    "percent_to_remove": 0.15, "n_elite": 2, "tournament_size": 4, "two_point_crossover_prob": 0.95
}

for folder, instances in instances_s_dict.items():
        for i in instances:
            print(f'Running instance {i}...')
            instance_pth = os.path.join(instances_s_pth, folder, i)
            vertices_w, edges = read_rakaj_graph_instance(instance_pth)

            results['instance'].append(folder)

            ga_results = []
            ga_time = []
            graph = Graph()
            graph.create_from_data(vertices_w, edges)
            
            for i in range(10):
                gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=100, **params)
                gen_algo.run()
                ga_results.append(gen_algo.previous_best_fitness)
                ga_time.append(gen_algo.running_time)
                print(f'Finished {i+1} of 10 in {gen_algo.running_time} s')

            results['ga_mean'].append(round(mean(ga_results),6))
            results['ga_max'].append(max(ga_results))
            results['ga_std'].append(round(std(ga_results), 6))
            results['ga_time'].append(round(mean(ga_time), 4))

            # start = time.time()
            # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
            # results['greedy'].append(greedy_w)
            # results['greedy_time'].append(round(time.time() - start, 4))
            
            # start = time.time()
            # ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
            # results['ilp'].append(ilp_w)
            # results['ilp_time'].append(round(time.time() - start, 4))
            results['greedy'].append(0)
            results['greedy_time'].append(0)
            results['ilp'].append(0)
            results['ilp_time'].append(0)


current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%H-%M_%d-%m-%Y")
output_csv = f'../results/new/small_results_{formatted_time}.csv'
output_params = f'../results/new/small_ga_params_{formatted_time}.json'

save_results_to_csv(results, output_csv)
save_params_to_json(output_params, params)

Running instance 0/Test/Problem.dat_50_50_0...
Finished 1 of 10 in 11.30378532409668 s
Finished 2 of 10 in 11.36411428451538 s
Finished 3 of 10 in 11.56542181968689 s
Finished 4 of 10 in 10.784418821334839 s
Finished 5 of 10 in 11.691102743148804 s
Finished 6 of 10 in 13.483954191207886 s
Finished 7 of 10 in 11.036206722259521 s
Finished 8 of 10 in 10.36756944656372 s
Finished 9 of 10 in 11.645784616470337 s


In [ ]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":300,  "population_size":1000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":60,
    "percent_to_remove": 0.15, "n_elite": 2, "tournament_size": 4, "two_point_crossover_prob": 0.95
}

for folder, instances in instances_m_dict.items():
        for i in instances:
            print(f'Running instance {i}...')
            instance_pth = os.path.join(instances_m_pth, folder, i)
            vertices_w, edges = read_rakaj_graph_instance(instance_pth)
            
            results['instance'].append(folder)

            ga_results = []
            ga_time = []
            graph = Graph()
            graph.create_from_data(vertices_w, edges)
            
            for i in range(10):
                gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=100, **params)
                gen_algo.run()
                ga_results.append(gen_algo.previous_best_fitness)
                ga_time.append(gen_algo.running_time)
                print(f'Finished {i+1} of 10 in {gen_algo.running_time} s')

            results['ga_mean'].append(round(mean(ga_results), 6))
            results['ga_max'].append(max(ga_results))
            results['ga_std'].append(round(std(ga_results), 6))
            results['ga_time'].append(round(mean(ga_time), 4))

            # start = time.time()
            # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
            # results['greedy'].append(greedy_w)
            # results['greedy_time'].append(round(time.time() - start, 4))
            
            # start = time.time()
            # ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
            # results['ilp'].append(ilp_w)
            # results['ilp_time'].append(round(time.time() - start, 4))
            results['greedy'].append(0)
            results['greedy_time'].append(0)
            results['ilp'].append(0)
            results['ilp_time'].append(0)

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%H-%M_%d-%m-%Y")
output_csv = f'../results/new/medium_results_{formatted_time}.csv'
output_params = f'../results/new/medium_ga_params_{formatted_time}.json'

save_results_to_csv(results, output_csv)
save_params_to_json(output_params, params)

In [ ]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":600,  "population_size":1000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":100,
    "percent_to_remove": 0.15, "n_elite": 2, "tournament_size": 4, "two_point_crossover_prob": 0.95
}

for folder, instances in instances_l_dict.items():
        for i in instances:
            print(f'Running instance {i}...')
            instance_pth = os.path.join(instances_l_pth, folder, i)
            vertices_w, edges = read_rakaj_graph_instance(instance_pth)

            results['instance'].append(folder)

            ga_results = []
            ga_time = []
            graph = Graph()
            graph.create_from_data(vertices_w, edges)
            
            for i in range(10):
                gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=200, **params)
                gen_algo.run()
                ga_results.append(gen_algo.previous_best_fitness)
                ga_time.append(gen_algo.running_time)
                print(f'Finished {i+1} of 10 in {gen_algo.running_time} s')

            results['ga_mean'].append(round(mean(ga_results), 6))
            results['ga_max'].append(max(ga_results))            
            results['ga_std'].append(round(std(ga_results), 6))
            results['ga_time'].append(round(mean(ga_time), 4))
            
            # start = time.time()
            # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
            # results['greedy'].append(greedy_w)
            # results['greedy_time'].append(round(time.time() - start, 4))
            
            # start = time.time()
            # ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
            # results['ilp'].append(ilp_w)
            # results['ilp_time'].append(round(time.time() - start, 4))
            results['greedy'].append(0)
            results['greedy_time'].append(0)
            results['ilp'].append(0)
            results['ilp_time'].append(0)

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%H-%M_%d-%m-%Y")
output_csv = f'../results/new/large_results_{formatted_time}.csv'
output_params = f'../results/new/large_ga_params_{formatted_time}.json'

save_results_to_csv(results, output_csv)
save_params_to_json(output_params, params)

# Testiranje na XLARGE instancama

In [ ]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":1500,  "population_size":2000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":300,
    "percent_to_remove": 0.15, "n_elite": 3, "tournament_size": 4, "two_point_crossover_prob": 0.95
}

for i in instances_xl_list:
    print(f'Running instance {i}...')
    instance_pth = os.path.join(instances_xl_pth, i)
    vertices_w, edges = read_graph_instance_networkrepo(instance_pth)

    results['instance'].append(i.replace('.mtx', ''))

    ga_results = []
    ga_time = []
    graph = Graph()
    graph.create_from_data(vertices_w, edges)

    for i in range(10):
        gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=500, **params)
        gen_algo.run()
        ga_results.append(gen_algo.previous_best_fitness)
        ga_time.append(gen_algo.running_time)
        print(f'Finished {i+1} of 10 in {gen_algo.running_time} s')


    results['ga_mean'].append(round(mean(ga_results),6))
    results['ga_max'].append(max(ga_results))
    results['ga_std'].append(round(std(ga_results), 6))
    results['ga_time'].append(round(mean(ga_time), 4))

    # start = time.time()
    # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
    # results['greedy'].append(greedy_w)
    # results['greedy_time'].append(round(time.time() - start, 4))
    results['greedy'].append(0)
    results['greedy_time'].append(0)
    
    try:
        start = time.time()
        ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
        results['ilp'].append(ilp_w)
        results['ilp_time'].append(round(time.time() - start, 4))
    except Exception as e:
        print(f'ILP failed with error: {e}')
        results['ilp'].append(0)
        results['ilp_time'].append(0)

current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%H-%M_%d-%m-%Y")
output_csv = f'../results/new/xlarge_results_{formatted_time}.csv'
output_params = f'../results/new/xlarge_ga_params_{formatted_time}.json'

save_results_to_csv(results, output_csv)
save_params_to_json(output_params, params)